In [1]:
# Simple parameters of this augmentation
OUTPUT_PATH = 'Augmented Datasets/'

import pandas as pd
import os

# Create the directory if it doesn't exist
if not os.path.exists(OUTPUT_PATH):   
    os.makedirs(OUTPUT_PATH)


import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# Load datasets
from datasets import load_dataset

fever_plus = load_dataset("tommasobonomo/sem_augmented_fever_nli")

c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from nltk.corpus import wordnet as wn


def get_synonym(entry, pos='NOUN'):

    for word in entry["wsd"]["hypothesis"]:
        if word["pos"]==pos:
            word_sense = wn.synset(word["nltkSynset"], pos=pos)
            #index =  
            print(word_sense)

            lemmas = word_sense.lemmas()
            for lemma in lemmas:
                # Return the first synonym different from the original word
                if lemma.name() != word_sense.name().split('.')[0]:
                    return lemma.name()  # Return the synonym
                
    return None  # Return None if no synonyms are found


for i in range(1000):
    print(get_synonym(fever_plus["train"][i], pos='NOUN'))

TypeError: WordNetCorpusReader.synset() got an unexpected keyword argument 'pos'

In [12]:
from nltk.corpus import wordnet as wn

def get_synonym(word, pos='n'):
    try:
        word_sense = wn.synset(word["nltkSynset"])
        lemmas = word_sense.lemmas()
        
        # Find a synonym that is not the original word
        for lemma in lemmas:
            if lemma.name() != word["text"]:  # Compare with original word text
                print(f"Replacing '{word['text']}' with '{lemma.name()}'")
                return lemma.name()  # Return the synonym
    except:
        return word["text"]  # Return the original word if any error occurs
    
    return word["text"]  # Return the original word if no synonym is found

def replace_nouns_with_synonyms(entry, pos='NOUN'):
    updated_hypothesis = []  # Collect updated words
    for word in entry["wsd"]["hypothesis"]:
        if word["pos"] == pos:  # Check if the word is a noun
            synonym = get_synonym(word, pos='n')
            updated_hypothesis.append(synonym)  # Replace with the synonym
        else:
            updated_hypothesis.append(word["text"])  # Keep the original word
    # Join the updated words into a single string hypothesis
    entry["hypothesis"] = ' '.join(updated_hypothesis)
    return entry

# Debug: Print the original hypotheses
for i in range(10):
    print("Original hypothesis:", ' '.join([w["text"] for w in fever_plus['train'][i]["wsd"]["hypothesis"]]))

print('-------------------------------')

# Apply the synonym replacement and print updated hypotheses
for i in range(10):
    updated_entry = replace_nouns_with_synonyms(fever_plus["train"][i], pos='NOUN')
    print("Updated hypothesis:", updated_entry['hypothesis'])


Original hypothesis: Roman Atwood is a content creator .
Original hypothesis: The Boston Celtics play their home games at TD Garden .
Original hypothesis: There is a movie called The Hunger Games .
Original hypothesis: Ryan Seacrest is a person .
Original hypothesis: Stranger than Fiction is a film .
Original hypothesis: Selena recorded music .
Original hypothesis: Selena recorded music .
Original hypothesis: Selena recorded music .
Original hypothesis: Selena recorded music .
Original hypothesis: John Wick : Chapter 2 was theatrically released in the Oregon .
-------------------------------
Updated hypothesis: Roman Atwood is a content creator .
Replacing 'home' with 'place'
Replacing 'games' with 'game'
Updated hypothesis: The Boston Celtics play their place game at TD Garden .
Replacing 'movie' with 'film'
Updated hypothesis: There is a film called The Hunger Games .
Replacing 'person' with 'individual'
Updated hypothesis: Ryan Seacrest is a individual .
Replacing 'film' with 'movie